# Temperature impact on Immigration

## Data Engineering Capstone Project

Project Summary

### The project follows the follow steps:

* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [ ]:
# import pandas as pd
import re
# import psycopg2  # For postgres
from collections import defaultdict
from datetime import datetime, timedelta
from pyspark.sql.functions import udf  # For pyspark
from pyspark.sql.types import *
from pyspark.sql.functions import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1625332603062_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Installs
#sc.install_pypi_package("pandas")
#sc.install_pypi_package("boto3")
#sc.install_pypi_package("psycopg2")
# sc.install_pypi_package("numpy==1.17.3")
#sc.install_pypi_package("smart_open")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version  
-------------------------- ---------
beautifulsoup4             4.9.3    
boto                       2.49.0   
click                      7.1.2    
jmespath                   0.10.0   
joblib                     1.0.1    
lxml                       4.6.2    
mysqlclient                1.4.2    
nltk                       3.5      
nose                       1.3.4    
numpy                      1.16.5   
pip                        9.0.1    
py-dateutil                2.2      
python37-sagemaker-pyspark 1.4.1    
pytz                       2021.1   
PyYAML                     5.4.1    
regex                      2021.3.17
setuptools                 28.8.0   
six                        1.13.0   
tqdm                       4.59.0   
wheel                      0.29.0   
windmill                   1.6

### Step 1: Scope the Project and Gather Data (Explore)

#### Scope 

The aim of this project is to enrich the US I94 immigration data with data such as demographics and temperature to have wider understanding of the immigration patterns. We will be creating dimension tables and one fact table. The data fromI94 is aggregated by destination city and then joined with temperature data, resulting in the one fact table we want. Ultimately, we can query on the databse to check if temperature affects the destination city for immigration.

#### Describe and Gather Data 

The I94 data which of the format SAS7BDAT(binary database storage format), is taken from [here](https://travel.trade.gov/research/reports/i94/historical/2016.html). 

World Temperature Data: This dataset came from Kaggle. You can read more about it [here](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data).

U.S. City Demographic Data: This data comes from OpenSoft. You can read more about it [here](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/).

Airport Code Table: This is a simple table of airport codes and corresponding cities. It comes from [here](https://datahub.io/core/airport-codes#data).

In [ ]:
# True indicates, it can be null
demographics_schema = StructType([
    StructField('City', StringType(), True),
    StructField('State', StringType(), True),
    StructField('Median Age', DoubleType(), True),
    StructField('Male Population', IntegerType(), True),
    StructField('Female Population', IntegerType(), True),
    StructField('Total Population', IntegerType(), True),
    StructField('Number of Veterans', IntegerType(), True),
    StructField('Foreign-born', IntegerType(), True),
    StructField('Average Household Size', DoubleType(), True),
    StructField('State Code', StringType(), True),
    StructField('Race', StringType(), True),
    StructField('Count', IntegerType(), True)
])
df_demographics = spark.read.csv("s3://capstonend/us-cities-demographics.csv",
                                 sep=";",
                                 schema=demographics_schema,
                                 header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# True indicates, it can be null
temperature_schema = StructType([
    StructField('dt', DateType(), True),
    StructField('AverageTemperature', DoubleType(), True),
    StructField('AverageTemperatureUncertainty', DoubleType(), True),
    StructField('City', StringType(), True),
    StructField('Country', StringType(), True),
    StructField('Latitude', StringType(), True),
    StructField('Longitude', StringType(), True)
])
#Temperature
temperature = 's3://capstonend/GlobalLandTemperaturesByCity.csv'
df_temperature = spark.read.csv(temperature,
                                schema=temperature_schema,
                                header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# True indicates, it can be null
airport_schema = StructType([
    StructField('ident', StringType(), True),
    StructField('type', StringType(), True),
    StructField('name', StringType(), True),
    StructField('elevation_ft', IntegerType(), True),
    StructField('continent', StringType(), True),
    StructField('iso_country', StringType(), True),
    StructField('iso_region', StringType(), True),
    StructField('municipality', StringType(), True),
    StructField('gps_code', StringType(), True),
    StructField('iata_code', StringType(), True),
    StructField('local_code', StringType(), True),
    StructField('coordinates', StringType(), True)
])
#Airport Codes
df_airport_codes = spark.read.csv("s3://capstonend/airport-codes_csv.csv",
                                  schema=airport_schema,
                                  header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
immigration = 's3://capstonend/sas_data/'
df_immigration = spark.read.parquet(immigration)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_demographics.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total Population: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)

In [ ]:
df_demographics.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+------------------+-----+
|         City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|              Race|Count|
+-------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+------------------+-----+
|Silver Spring|     Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|Hispanic or Latino|25924|
|       Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|             White|58723|
+-------------+-------------+----------+---------------+-----------------+-----------

In [ ]:
df_temperature.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- dt: date (nullable = true)
 |-- AverageTemperature: double (nullable = true)
 |-- AverageTemperatureUncertainty: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)

In [ ]:
df_temperature.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+-----------------------------+-----+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
only showing top 2 rows

In [ ]:
df_airport_codes.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)

In [ ]:
df_airport_codes.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
only showing top 2 rows

In [ ]:
df_immigration.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [ ]:
df_immigration.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|         admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+
|1817663.0|2016.0|   4.0| 148.0| 131.0|    SFR|20554.0|    1.0|     CA|20560.0|  23.0|    2.0|  1.0|20160410|    null| null|      G|      O|   null|      M| 1993.0|07082016|     F|  null|     LX|5.5960415633E10|00038|      WT|
|1817664.0|2016.0|   4.0| 148.0| 131.0|    SFR|20554.0|    1.0|     CA|20561.0|  29.0|    1.

## Cleaning (Transform)

In [ ]:
df_demographics = df_demographics.dropna()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Temperature Data - We will drop all data points where AverageTemperature is NaN, duplicate locations, and add the i94port of the location in each entry.

In [ ]:
# df_temperature = df_temperature.dropna()

# Filter out data points with NaN average temperature
df_temperature = df_temperature.filter(
    df_temperature.AverageTemperature != 'NaN')
df_temperature = df_temperature.dropDuplicates(['City', 'Country'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
sc.install_pypi_package("boto3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

*I94 immigration data* - We wil drop all data points with the destination city code i94port is not a valid value (XXX, 99, NaN, etc). This is described in I94_SAS_Labels_Description.SAS

In [ ]:
import boto3

s3 = boto3.resource('s3')
bucket = 'capstonend'
bucket = s3.Bucket(bucket)
file = bucket.objects.all()
for f in file:
    if f.key == 'i94port_valid.txt':
        print(f.key)
        txt = f.get()['Body'].read()
lines = str(txt).split('\\n')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

i94port_valid.txt

In [ ]:
# Create dictionary of valid i94port codes
re_obj = re.compile(r'\'(.*)\'.*\'(.*)\'')
i94port_valid = {}
for line in lines:
    match = re_obj.search(line)
    i94port_valid[match[1]] = [match[2].strip()]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#print(i94port_valid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Clean I94 immigration data
def clean_i94_data(file):
    '''    
    Input: Path to I94 immigration data file
    Output: Spark dataframe of I94 immigration data with valid i94port
    '''
    global df_immigration
    # Filter out entries where i94port is invalid
    df_immigration = df_immigration.filter(
        df_immigration.i94port.isin(list(i94port_valid.keys())))
    return df_immigration

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_immigration = clean_i94_data(df_immigration)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_immigration.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SEO| null|      G|   null|      Y|   null| 1991.0|     D/S|     M|  null|   null|  3.73679633E9|00296|      F1|
| 15.0|2016.0|   4.0| 101.0| 101.0|    WAS|20545.0|    1.0|     MI|20691.0|  55.0|    2.0|  1.0|20160401|    nul

In [ ]:
@udf()
def get_i94port(city):
    '''
    Input: City name
    Output: Corresponding i94port
    
    '''
    for key in i94port_valid:
        if city and city.lower() in i94port_valid[key][0].lower():
            return key

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Add iport94 code based on city name, helps in joins
df_temperature = df_temperature.withColumn("i94port",
                                           get_i94port(df_temperature.City))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Remove data points with no iport94 code
df_temperature = df_temperature.filter(df_temperature.i94port != 'null')
df_temperature.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+-----------------------------+----------------+--------------+--------+---------+-------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|            City|       Country|Latitude|Longitude|i94port|
+----------+------------------+-----------------------------+----------------+--------------+--------+---------+-------+
|1824-01-01|            25.229|                        1.094|           Anaco|     Venezuela|   8.84N|   64.05W|    ANA|
|1820-01-01|              9.89|           3.8819999999999997|  Corpus Christi| United States|  28.13N|   97.27W|    CRP|
|1855-05-01|            10.468|                        1.598|     Los Angeles|         Chile|  37.78S|   73.22W|    LOS|
|1825-01-01|            14.028|                        2.821|       Monterrey|        Mexico|  26.52N|  100.30W|    MTY|
|1743-11-01|             3.264|                        1.665|          Newark| United States|  40.99N|   74.56W|    NEW|
|1852-07-01|            15.488| 

In [ ]:
df_airport_codes = df_airport_codes.dropna()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_temperature.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

207

In [ ]:
df_temperature = df_temperature.filter(
    df_temperature.Country == 'United States')
df_temperature.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

112

## Define the Data Model (Load)

Fact Table - This will contain information from the I94 immigration data joined with the city temperature data on i94port
Columns:
i94yr = year, i94mon = numeric month, i94cit = 3 digit code of origin city, i94port = 3 character code of destination city, arrdate = arrival date, i94mode = 1 digit travel code, depdate = departure date, i94visa = reason for immigration, AverageTemperature = average temperature of destination city

Dimension Tables - df_immigration, df_temperature

df_immigration:-
i94yr = year, i94mon = numeric month, i94cit = 3 digit code of origin city, i94port = 3 character code of destination city, arrdate = arrival date, i94mode = 1 digit travel code, depdate = departure date, i94visa = reason for immigration

df_temperature:-
i94port = code of destination city (mapped from cleaned up immigration data), AverageTemperature = average temperature, City = city name, Country = country name, Latitude= latitude, Longitude = longitude

In [ ]:
# Immigration, extract and save
immigration_table = df_immigration.select([
    "i94yr", "i94mon", "i94cit", "i94port", "arrdate", "i94mode", "depdate",
    "i94visa"
])
immigration_table.write.mode("append").partitionBy("i94port").parquet(
    "immigration.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Temperature
temp_table = df_temperature.select([
    "AverageTemperature", "City", "Country", "Latitude", "Longitude", "i94port"
])
temp_table.write.mode("append").partitionBy("i94port").parquet(
    "temperature.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_demographics = df_demographics.withColumnRenamed(
    'State Code', 'State_Code').withColumnRenamed(
        'Male Population', 'Male_Population').withColumnRenamed(
            'Female Population', 'Female_Population').withColumnRenamed(
                'Total Population', 'Total_Population').withColumnRenamed(
                    'Number of Veterans',
                    'Number_of_Veterans').withColumnRenamed(
                        'Average Household Size',
                        'Average_Household_Size').withColumnRenamed('Median Age', 'Median_Age')
df_demographics.write.mode("append").parquet("demographics.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# For fact table, create views to use sql for easy join
df_immigration.createOrReplaceTempView("immigration_view")
df_temperature.createOrReplaceTempView("temp_view")

# Create the fact table by joining the immigration and temperature views
fact_table = spark.sql('''
SELECT immigration_view.i94yr as year,
       immigration_view.i94mon as month,
       immigration_view.i94cit as city,
       immigration_view.i94port as i94port,
       immigration_view.arrdate as arrival_date,
       immigration_view.depdate as departure_date,
       immigration_view.i94visa as reason,
       temp_view.AverageTemperature as temperature,
       temp_view.Latitude as latitude,
       temp_view.Longitude as longitude
FROM immigration_view
JOIN temp_view ON (immigration_view.i94port = temp_view.i94port)
''')
fact_table.write.mode("append").partitionBy("i94port").parquet(
    "fact_table.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Quality Checks

In [ ]:
def quality_checks(table,name_):
  '''
  Input: table, and name of the table
  Output: Print statement to check if the table passed the quality checks 
  '''
  c = table.count()
  if c > 0:
    print(f'No. of rows in {name_} is {c}')
  else:
    print(f'Data quality check failed for {name_}')

In [ ]:
quality_checks(df_immigration,'df_immigration')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'No. of rows in df_immigration is 3088544'

In [ ]:
quality_checks(df_temperature,'df_temperature')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'No. of rows in df_temperature is 112'

In [ ]:
quality_checks(df_demographics,'df_demographics')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'No. of rows in df_demographics is 2875'

In [ ]:
quality_checks(fact_table,'fact_table')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'No. of rows in fact_table is 2568134'

In [ ]:
# Add iport94 code based on city name
df_temperature = df_temperature.withColumn("i94port",
                                           get_i94port(df_temperature.City))
# Remove data points with no iport94 code
df_temperature = df_temperature.filter(df_temperature.i94port != 'null')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In this project, we used Spark since it can easily handle multiple file formats (parquet, csv, text) that contain large amounts of data. Spark SQL was used form fact table.

Since, files update monthly, we can create airflow pipeline to run monthly if needed.